In [34]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.insert(1, '../../SourceCode/')
import DaedalusGlobals as DaedalusGlobals

import ipywidgets as w
from netCDF4 import Dataset 
import time

import OrbitFileConversions

style1 = {'description_width':'120px'}
layout1 = {'width':'780px'}
OrbitFilename_Text     = w.Text(value="", description='Orbit file name:', style=style1, layout=layout1)
TIEGCMfilesFolder_Text = w.Text(value=DaedalusGlobals.TIEGCM_Files_Path + "TIEGCM_DERIVED_PRODUCTS/", description='TIEGCM files folder:', style=style1, layout=layout1)
ResultFilename_Text    = w.Text(value="", description='Result file name:', style=style1, layout=layout1)
Regions_Dropdown       = w.Dropdown( options=["AEM", "AFM", "AEE", "AED", "EEJ", "EPB", "SQ", "CF", "PCF"], description='Region of interest: ', style=style1, layout=layout1)

def Convert_AddFieldsToCSVorbit_Btn_Clicked( b ):
    print( "Reading ", OrbitFilename_Text.value, "and Writing", ResultFilename_Text.value  )
    OrbitFileConversions.ApexConvert( OrbitFilename_Text.value, ResultFilename_Text.value )
    print( "Done" )

def Convert_CherrypickFromTIEGCMfiles_Btn_Clicked( b ):
    # initialize
    print( "Reading from", TIEGCMfilesFolder_Text.value, "and Writing", ResultFilename_Text.value  )
    startSecs = time.time()
    ResultFileIdx = 0
    
    # define limits of the selected region
    minMLT=22; maxMLT=2;  minMagLat=60; maxMagLat=75

    # Create the result file
    resultCDF = Dataset( ResultFilename_Text.value, 'w' )
    resultCDF.createDimension("time", None)
    v = resultCDF.createVariable("time","u8",("time",))
    v.units = "minutes"
    v = resultCDF.createVariable("lat","f8",("time",))
    v.description = ""
    v.units = "degrees"
    v = resultCDF.createVariable("lon","f8",("time",)) 
    v.description = ""
    v.units = "degrees"
    v = resultCDF.createVariable("MagLat","f8",("time",))
    v.description = "Magnetic Latitude"
    v.units = "degrees"
    v = resultCDF.createVariable("MLT","f8",("time",))
    v.description = "Mean Local Time"
    v.units = "hours"
    v = resultCDF.createVariable("Altitude","f8",("time",))
    v.units = "km"
    v = resultCDF.createVariable("Kp","f8",("time",))
    v.description = "Kp index"
    v.units = "-"    
    v = resultCDF.createVariable("Joule_Heating","f8",("time",))
    v.description = "Joule Heating"
    v.units = "W/m3"
    
    
    
    # Read all files in the TIEGCM folder - we will process them all
    DataFilesPath = TIEGCMfilesFolder_Text.value    
    AllDataFiles = sorted( glob.glob( DataFilesPath + "*.nc" ) )
    for currentDataFile in AllDataFiles:
        # Read file from disk into memory
        print( "Reading", currentDataFile )
        try:
            CDFroot = Dataset( currentDataFile, 'r' )
        except:
            print ( "WRONG FORMAT:", currentDataFile )
            continue
        length_time = CDFroot.variables['Joule Heating'].shape[0]
        length_lev  = CDFroot.variables['Joule Heating'].shape[1]
        length_lat  = CDFroot.variables['Joule Heating'].shape[2]
        length_lon  = CDFroot.variables['Joule Heating'].shape[3]        
        print ("Loading file into memory")
        TIMEs   = CDFroot.variables[         'time'][:, :, :, :]   
        LATs    = CDFroot.variables[          'lat'][:, :, :, :]   
        LONs    = CDFroot.variables[          'lon'][:, :, :, :]   
        MAGLATs = CDFroot.variables[         'MLAT'][:, :, :, :]   
        MLTs    = CDFroot.variables[          'MLT'][:, :, :, :]
        ALTs    = CDFroot.variables[           'ZG'][:, :, :, :] / 100000
        JHs     = CDFroot.variables['Joule Heating'][:, :, :, :]
        KPs     = CDFroot.variables[           'KP'][:]
        
        for idx in range(0, length_time):        
            includeIt = False
            if includeIt:                 
                resultCDF.variables[         "time"][ResultFileIdx] = TIMES[ idx ]
                resultCDF.variables[          "lat"][ResultFileIdx] = LATs[ idx ]
                resultCDF.variables[          "lon"][ResultFileIdx] = LONs[ idx ]
                resultCDF.variables[       "MagLat"][ResultFileIdx] = MAGLATs[ idx ]
                resultCDF.variables[          "MLT"][ResultFileIdx] = MLTs[ idx ]
                resultCDF.variables[     "Altitude"][ResultFileIdx] = ALTs[ idx ]
                resultCDF.variables[           "Kp"][ResultFileIdx] = KPs[ idx ]
                resultCDF.variables["Joule_Heating"][ResultFileIdx] = JHs[ idx ]
    
    # close files, tide up
    resultCDF.close()
    finishSecs = time.time()   
    print( "Finished in", finishSecs-startSecs, "seconds" )
    
    
def createGUI():
    MainTab   = w.Tab() 
    #
    CherrypickFromTIEGCMfiles_Panel = w.VBox()    
    CherrypickFromTIEGCMfiles_Label = w.HTML(value="<font color='goldenrod'>NOT VALID YET: This application reads several orbit files, chooses data according to the ranges of the region and saves them in a single file.<br>Please select a folder containing orbit files and the result filename and press 'Convert':")
    Convert_CherrypickFromTIEGCMfiles_Btn = w.Button (description='Convert',tooltip="Convert",)
    Convert_CherrypickFromTIEGCMfiles_Btn.style.button_color = 'Coral'
    Convert_CherrypickFromTIEGCMfiles_Btn.on_click( Convert_CherrypickFromTIEGCMfiles_Btn_Clicked )
    CherrypickFromTIEGCMfiles_Panel.children = [CherrypickFromTIEGCMfiles_Label, TIEGCMfilesFolder_Text, Regions_Dropdown, ResultFilename_Text, Convert_CherrypickFromTIEGCMfiles_Btn]    
    #
    AddFieldsToCSVorbit_Panel = w.VBox()    
    AddFieldsToCSVorbit_Label = w.HTML(value="<font color='goldenrod'>This application adds the following fields to a csv orbit file:<br>Daedalus.Magnetic Latitude, Daedalus.Magnetic Longitude, Daedalus.MLT.<br>Please type the orbit filename and the result filename and press 'Convert':")
    Convert_AddFieldsToCSVorbit_Btn = w.Button (description='Convert',tooltip="Convert",)
    Convert_AddFieldsToCSVorbit_Btn.style.button_color = 'MediumTurquoise'
    Convert_AddFieldsToCSVorbit_Btn.on_click( Convert_AddFieldsToCSVorbit_Btn_Clicked )
    AddFieldsToCSVorbit_Panel.children = [AddFieldsToCSVorbit_Label, OrbitFilename_Text, ResultFilename_Text, Convert_AddFieldsToCSVorbit_Btn]
    #
    MainTab.children = [ CherrypickFromTIEGCMfiles_Panel,    AddFieldsToCSVorbit_Panel ]
    MainTab.set_title(0, 'Cherrypick from orbit')
    MainTab.set_title(1, 'Add fields to orbit')
    #
    return MainTab


ResultFilename_Text.value   = DaedalusGlobals.TIEGCM_Files_Path + "TIEGCM_DERIVED_PRODUCTS_partitioned/" + "AEM.nc"

createGUI()    


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
